In [ ]:
!ls

In [ ]:
# Step 1: Install required libraries (run once)
# pip install diffusers transformers accelerate torch pillow

import torch
from diffusers import StableDiffusionPipeline
import os

# Step 2: Load Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Step 3: Create output directory
output_dir = "synthetic_image_dataset"
os.makedirs(output_dir, exist_ok=True)

# Step 4: Input prompts
prompts = [
    "A futuristic city with flying cars at sunset",
    "A cute robot studying in a classroom",
    "A realistic portrait of an Indian farmer smiling",
    "A cyberpunk street at night with neon lights",
    "A peaceful mountain landscape with river and trees"
]

# Step 5: Generate and save images
for i, prompt in enumerate(prompts):
    image = pipe(prompt).images[0]
    image.save(f"{output_dir}/image_{i+1}.png")

print("Synthetic dataset generated successfully!")


In [ ]:
cat_species = [
    "Abyssinian", "American Bobtail", "American Curl", "Balinese",
    "Bengal", "Birman", "Bombay", "British Shorthair",
    "Burmese", "Chartreux", "Cornish Rex", "Devon Rex",
    "Egyptian Mau", "Exotic Shorthair", "Himalayan", "Japanese Bobtail",
    "Korat", "LaPerm", "Maine Coon", "Manx",
    "Norwegian Forest Cat", "Ocicat", "Oriental Shorthair", "Persian",
    "Peterbald", "Pixie-bob", "Ragdoll", "Russian Blue",
    "Savannah", "Scottish Fold", "Selkirk Rex", "Siamese",
    "Siberian", "Singapura", "Snowshoe", "Somali",
    "Sphynx", "Tonkinese", "Turkish Angora", "Turkish Van",
    "American Shorthair"
]


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import os

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")
pipe.enable_attention_slicing()

base_dir = "cat_dataset"
os.makedirs(base_dir, exist_ok=True)

images_per_class = 10

for species in cat_species:
    species_dir = os.path.join(base_dir, species)
    os.makedirs(species_dir, exist_ok=True)

    for i in range(images_per_class):
        prompt = f"A realistic photo of a {species} cat, high detail, natural lighting"
        image = pipe(
            prompt=prompt,
            negative_prompt="blurry, cartoon, distorted",
            num_inference_steps=20,
            guidance_scale=7.5
        ).images[0]

        image.save(f"{species_dir}/{species}_{i+1}.png")

    print(f"Generated images for {species}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
print(os.listdir("/content"))


In [ ]:
import shutil
import os

source = "/content/cat_dataset"
destination = "/content/drive/MyDrive/cat_datasets/cat_dataset"

os.makedirs("/content/drive/MyDrive/cat_datasets", exist_ok=True)

shutil.copytree(source, destination)

print("✅ Dataset saved to Google Drive successfully!")


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder("cat_dataset", transform=transform)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(pretrained=True)

# Replace final layer for 41 classes
model.fc = nn.Linear(model.fc.in_features, 41)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 5   # Enough for lab

for epoch in range(epochs):
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
save_path = "/content/drive/MyDrive/cat_models/cat_species_resnet18.pth"
import os
os.makedirs("/content/drive/MyDrive/cat_models", exist_ok=True)

torch.save(model.state_dict(), save_path)
print("Model saved to Google Drive!")


In [ ]:
# torch.save(model.state_dict(), "cat_species_resnet18.pth")
# print("Model saved successfully!")


In [ ]:
from PIL import Image

img = Image.open("/content/Abyssian.avif")
img = transform(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(img)
    pred = torch.argmax(output, dim=1)

print("Predicted Class:", dataset.classes[pred.item()])


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder("/content/cat_dataset", transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
correct = 0
total = 0

model.eval()   # evaluation mode

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")


In [ ]:
import torch.nn.functional as F
from PIL import Image

img = Image.open("/content/cat_dataset/Abyssinian/Abyssinian_1.png")
img = transform(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(img)
    probabilities = F.softmax(output, dim=1)
    confidence, pred = torch.max(probabilities, dim=1)

print("Predicted Class:", dataset.classes[pred.item()])
print("Confidence:", confidence.item() * 100, "%")


In [ ]:
from PIL import Image

# Load local image file
img_path = "/content/cat_dataset/Russian Blue/Russian Blue_1.png"   # make sure spelling & path is correct
img = Image.open(img_path).convert("RGB")

img


In [ ]:
img_tensor = transform(img).unsqueeze(0).to(device)


In [ ]:
import torch.nn.functional as F

model.eval()

with torch.no_grad():
    output = model(img_tensor)
    probs = F.softmax(output, dim=1)
    confidence, pred = torch.max(probs, dim=1)

predicted_class = dataset.classes[pred.item()]

print("Predicted Class:", predicted_class)
print("Confidence:", f"{confidence.item()*100:.2f}%")
